In [2]:
from settings import config

DATA_DIR = config("DATA_DIR")  # Fetch the data directory path
df_ois_wide = pd.read_excel(f"{DATA_DIR}/OIS.xlsx", sheet_name=0)
df_fut_wide = pd.read_excel(f"{DATA_DIR}/treasury_spot_futures.xlsx", sheet_name=0)



In [4]:
df_fut_wide

,Start Date,2000-01-01 00:00:00,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40
0,End Date,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,10 year,NaN,NaN,NaN,5 year,NaN,NaN,NaN,2 year,...,NaN,NaN,20 Year,NaN,NaN,NaN,30 Year,NaN,NaN,NaN
2,Nearby,TY1 Comdty,NaN,NaN,NaN,FV1 Comdty,NaN,NaN,NaN,TU1 Comdty,...,NaN,NaN,US2 Comdty,NaN,NaN,NaN,WN2 Comdty,NaN,NaN,NaN
3,NaN,Implied Repo Rate of CTD Bond,Volume,NaN,NaN,Implied Repo Rate of CTD Bond,Volume,NaN,NaN,Implied Repo Rate of CTD Bond,...,NaN,NaN,Implied Repo Rate of CTD Bond,Volume,NaN,NaN,Implied Repo Rate of CTD Bond,Volume,NaN,NaN
4,Dates,FUT_IMPLIED_REPO_RT,PX_VOLUME,CURRENT_CONTRACT_MONTH_YR,PX_Last,FUT_IMPLIED_REPO_RT,PX_VOLUME,CURRENT_CONTRACT_MONTH_YR,PX_Last,FUT_IMPLIED_REPO_RT,...,CURRENT_CONTRACT_MONTH_YR,PX_Last,FUT_IMPLIED_REPO_RT,PX_VOLUME,CURRENT_CONTRACT_MONTH_YR,PX_Last,FUT_IMPLIED_REPO_RT,PX_VOLUME,CURRENT_CONTRACT_MONTH_YR,PX_Last
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6304,2024-02-23 00:00:00,4.50768,4246944,MAR 24,109.875,3.71846,4210442,MAR 24,106.476562,2.01884,...,JUN 24,102.359375,4.72078,490537,JUN 24,119.09375,3.40785,772555,JUN 24,127.71875
6305,2024-02-26 00:00:00,4.51815,5341315,MAR 24,109.609375,3.72569,4813722,MAR 24,106.296875,1.98446,...,JUN 24,102.269531,4.66795,836284,JUN 24,118.625,3.48949,946552,JUN 24,126.9375
6306,2024-02-27 00:00:00,4.3875,2419680,MAR 24,109.53125,3.67458,2293032,MAR 24,106.265625,1.6437,...,JUN 24,102.257812,4.67937,589937,JUN 24,118.3125,3.37423,484927,JUN 24,126.40625
6307,2024-02-28 00:00:00,4.54494,486902,MAR 24,109.8125,3.38666,521452,MAR 24,106.476562,1.53455,...,JUN 24,102.378906,4.51751,442507,JUN 24,118.75,3.39269,332881,JUN 24,127.09375
